# Different methods for merging data frames

pandas comes with a function merge to join datasets. It exists also a `join` method which is built on top of `merge`. It won't be discussed here. We will see here the various methods we can use in pandas to replicate SQL joins.

All pictures in this notebook are taken from Christopher-Moffatt (see https://www.codeproject.com/Articles/33052/Visual-Representation-of-SQL-Joins). This article explains the SQL syntax for each type of join. This notebook can be seen as a translation of his work with pandas data frames.

## Create datasets

Let's create two datasets to play with:

In [1]:
import pandas as pd

In [2]:
df1 = pd.DataFrame({"City": ["Paris", "London", "New York", "Tokyo"],
                    "Population": [12475808, 12317800, 20182305, 42794714],
                    "Country": ["France", "UK", "USA", "Japan"]})
df1.head()

,City,Population,Country
0,Paris,12475808,France
1,London,12317800,UK
2,New York,20182305,USA
3,Tokyo,42794714,Japan


In [3]:
df2 = pd.DataFrame({"Country": ["France", "UK", "USA", "China"],
                    "Population": [67595000, 65110000, 324811000, 1376049000]})
df2.head()

,Country,Population
0,France,67595000
1,UK,65110000
2,USA,324811000
3,China,1376049000


## Different types of SQL joins

<img src="img/Visual_SQL_JOINS_orig.jpg">

## Inner join

<img src="img/INNER_JOIN.png">

In [4]:
result = pd.merge(df1, df2, on="Country", suffixes=("_city", "_country"))
result.head()

,City,Population_city,Country,Population_country
0,Paris,12475808,France,67595000
1,London,12317800,UK,65110000
2,New York,20182305,USA,324811000


By default, `merge` is doing an inner join: only matching rows are kept. The `on` attribute enables to specify on which column the join is done. If both columns do not have the same name you can use `left_on` and `right_on` instead.

As we have one column (Population) that exists in both data frames, it is possible to provide a `suffixes` attribute to make column names unique in the data frame resulting from `merge`.

We can make this more explicit by using the `how` attribute and set its value to inner. This does not change anything as it is the default value.

In [5]:
result = pd.merge(df1, df2, on="Country", suffixes=("_city", "_country"), how='inner')
result.head()

,City,Population_city,Country,Population_country
0,Paris,12475808,France,67595000
1,London,12317800,UK,65110000
2,New York,20182305,USA,324811000


## Left join

<img src="img/LEFT_JOIN.png">

In [6]:
result = pd.merge(df1, df2, on="Country", suffixes=("_city", "_country"), how='left')
result.head()

,City,Population_city,Country,Population_country
0,Paris,12475808,France,67595000.0
1,London,12317800,UK,65110000.0
2,New York,20182305,USA,324811000.0
3,Tokyo,42794714,Japan,NaN


This time all rows from the first data frames are kept, even if no matching row has been found in the second data frame. Unmatched rows have NaN values for the columns coming from the second dataset.

## Right join

<img src="img/RIGHT_JOIN.png">

In [7]:
result = pd.merge(df1, df2, on="Country", suffixes=("_city", "_country"), how='right')
result.head()

,City,Population_city,Country,Population_country
0,Paris,12475808.0,France,67595000
1,London,12317800.0,UK,65110000
2,New York,20182305.0,USA,324811000
3,NaN,NaN,China,1376049000


The right join does the opposite. All rows from the second data frame are kept even if no match is found. 

## Outer join

<img src="img/FULL_OUTER_JOIN.png">

In [8]:
result = pd.merge(df1, df2, on="Country", suffixes=("_city", "_country"), how='outer')
result.head()

,City,Population_city,Country,Population_country
0,Paris,12475808.0,France,6.759500e+07
1,London,12317800.0,UK,6.511000e+07
2,New York,20182305.0,USA,3.248110e+08
3,Tokyo,42794714.0,Japan,NaN
4,NaN,NaN,China,1.376049e+09


Here, every row are kept whatever their origin. Unmatched rows in other data frame report NaN values.

## Outer excluding join

<img src="img/OUTER_EXCLUDING_JOIN.png">

For the next series of joins, we use an additional argument `indicator` which adds a column in the resulting data frame regarding the merge operation. the value of this column is either:

- both: rows have been matched
- left_only: row is unmatched and only appears in the first data frame
- right_only: row is unmatched and only appears in the second data frame

In [9]:
result = pd.merge(df1, df2, on="Country", suffixes=("_city", "_country"), how='outer', indicator=True)
result.head()

,City,Population_city,Country,Population_country,_merge
0,Paris,12475808.0,France,6.759500e+07,both
1,London,12317800.0,UK,6.511000e+07,both
2,New York,20182305.0,USA,3.248110e+08,both
3,Tokyo,42794714.0,Japan,NaN,left_only
4,NaN,NaN,China,1.376049e+09,right_only


With this new column it is easy to filter out the matched rows resulting from the outer join to have an outer excluding join:

In [10]:
result = pd.merge(df1, df2, on="Country", suffixes=("_city", "_country"), how='outer', indicator=True)
result = result[result["_merge"] != "both"]
result.head()

,City,Population_city,Country,Population_country,_merge
3,Tokyo,42794714.0,Japan,NaN,left_only
4,NaN,NaN,China,1.376049e+09,right_only


Only unmatched rows are kepts, whatever their origin.

## Left excluding join

<img src="img/LEFT_EXCLUDING_JOIN.png">

In [11]:
result = pd.merge(df1, df2, on="Country", suffixes=("_city", "_country"), how='left', indicator=True)
result = result[result["_merge"] == "left_only"]
result.head()

,City,Population_city,Country,Population_country,_merge
3,Tokyo,42794714,Japan,NaN,left_only


This only keeps the unmatched row from the first data frame.

## Right excluding join

<img src="img/RIGHT_EXCLUDING_JOIN.png">

In [12]:
result = pd.merge(df1, df2, on="Country", suffixes=("_city", "_country"), how='right', indicator=True)
result = result[result["_merge"] == "right_only"]
result.head()

,City,Population_city,Country,Population_country,_merge
3,NaN,NaN,China,1376049000,right_only


This only keeps the unmatched row from the second data frame.